In [100]:
from petlib.ec import EcGroup
from petlib.pack import encode
from hashlib import sha256
from zksk import Secret, DLRep

# Initialize an elliptic curve group
G = EcGroup(713)

# Key generation for group manager and members
x0 = G.order().random()
g = G.generator()
h0 = g.pt_mul(x0)

m = 12

# Public keys of group members
private_keys = [G.order().random() for _ in range(m)]
public_keys = [g.pt_mul(x) for x in private_keys]

def generate_signature(member_idx, message):
    """Generates a group signature on a message by a member."""
    xi = private_keys[member_idx]
    u = G.order().random()
    A = g.pt_mul(u)
    B = h0.pt_mul(u) + g.pt_mul(xi)
    r = Secret(xi)
    stmt = DLRep(A, r * g) & DLRep(B, r * h0)
    zk_proof = stmt.prove()
    commit = sha256(encode(message) + zk_proof.serialize())
    return A, B, zk_proof, commit

def verify_signature(A, B, zk_proof, commit, message):
    """Verifies a group signature on a message."""
    print("\tverify_signature")
    recomputed_commit = sha256(encode(message) + zk_proof.serialize())
    assert commit.digest() == recomputed_commit.digest()
    print("If you see it - verification is correct")

def open_signature(A, B, member_idx, message):
    """Opens a group signature to reveal which member signed a message."""
    print("\topen_signature")
    xi = private_keys[member_idx]
    A_inv = A.pt_mul(x0).pt_mul(-1)
    decrypted = B + A_inv
    hi = public_keys[member_idx]
    if decrypted == hi:
        print("Signature is valid and opened.")
    else:
        print("Invalid signature or opening.")

# Example usage
message = b"Hello, World!"
member_idx = 5
A, B, zk_proof, response = generate_signature(member_idx, message)
verify_signature(A, B, zk_proof, response, message)
open_signature(A, B, member_idx, message)

	verify_signature
If you see it - verification is correct
	open_signature
Signature is valid and opened.
